## Neutral Network Model for CA Housing Dataset

#### 1. Prepare Data
Model specific pre-processing, including scaling data for neural network.

In [34]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Import Datasets
train = pd.read_csv('datasets/train.csv')
val = pd.read_csv('datasets/val.csv')
test = pd.read_csv('datasets/test.csv')

# Select the columns to scale (last two columns)
cols_to_scale = ['median_income', 'median_house_value']

# Initialize scaler and fit on training data only
scaler = StandardScaler()
train_scaled_values = scaler.fit_transform(train[cols_to_scale])
val_scaled_values = scaler.transform(val[cols_to_scale])
test_scaled_values = scaler.transform(test[cols_to_scale])

# Create new column names
scaled_col_names = [f"{col}_scaled" for col in cols_to_scale]

# Add scaled columns and drop originals
for df, scaled_data in zip([train, val, test], [train_scaled_values, val_scaled_values, test_scaled_values]):
    for orig_col, scaled_col in zip(cols_to_scale, scaled_col_names):
        df[scaled_col] = scaled_data[:, scaled_col_names.index(scaled_col)]
    df.drop(columns=cols_to_scale, inplace=True)

#### Define Model Parameters
Define Model and Hyperparameter Set
